In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install imageio

In [3]:
!pip install tensorlayer

  Using cached https://files.pythonhosted.org/packages/9a/f6/3b3c82c5c75cae471e02fb584136168d732e17ae9db2d21c5dc82f9790f8/Pillow-5.3.0-cp27-cp27mu-manylinux1_x86_64.whl
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [4]:
!pip install Pillow==4.0.0
!pip install PIL
!pip install image


  Using cached https://files.pythonhosted.org/packages/89/99/0e3522a9764fe371bf9f7729404b1ef7d9c4fc49cbe5f1761c6e07812345/Pillow-4.0.0-cp27-cp27mu-manylinux1_x86_64.whl
scikit-image 0.14.1 has requirement pillow>=4.3.0, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.misc
import matplotlib.pyplot as plt
import cv2
import os
import re
import imageio
import pickle
from PIL import Image
IMAGE_SZ = 128 
PATCH1_SIZE=64
NUMBER1=128//64
PATCH2_SIZE=8
NUMBER2=128//8
Label_size=32
half_label_size=8


#loads image and crops it into 128X128
def load_city_image(image):
    im = Image.open(image).convert('RGB')
    width, height = im.size
    print(im.size)
    left = (width - IMAGE_SZ) / 2
    top = (height - IMAGE_SZ) / 2
    im = im.crop((left, top, left + IMAGE_SZ, top + IMAGE_SZ))
    pix = np.array(im)
    assert pix.shape == (IMAGE_SZ, IMAGE_SZ, 3)
    return im

#Global patch:  centred crop 128X64, 
def Patch_0(im,Label_size):
    a=[]
    im=im.crop((32,0,96,IMAGE_SZ))
    for i in range(Label_size):
                a.append(np.array(im))
    pix=np.array(im)
    return a

def Patch(image,PATCH_SIZE,NUMBER,Label_size):
    count= Label_size//(2*NUMBER)
    print(type(image))
    a=[]
    for i in range(2):
        for j in range(NUMBER):
            left=32+i*64
            top=(PATCH_SIZE/2)+j*PATCH_SIZE
            if i==0:
                im = image.crop((left, top-PATCH_SIZE/2, left + PATCH_SIZE/2, top + PATCH_SIZE/2))
            elif i==1:
                im = image.crop((left-PATCH_SIZE/2, top-PATCH_SIZE/2, left , top + PATCH_SIZE/2))
            for k in range(count):
                a.append(np.array(im)) 
    return a

def Patch_label(image,PATCH_SIZE,NUMBER,Label_size):
    count= Label_size//(2*NUMBER)
    print(type(image))
    a=[]
    for i in range(2):
        for j in range(NUMBER):
            left=32+i*64
            top=(PATCH_SIZE/2)+j*PATCH_SIZE
            im = image.crop((left-PATCH_SIZE/2, top-PATCH_SIZE/2, left + PATCH_SIZE/2, top + PATCH_SIZE/2))
            for k in range(count):
                pix=np.array(im)
                a.append(pix) 
    return a

#if training
DATA="/content/drive/My Drive/NN&RL/coast"
#if testing
#DATA="/content/drive/My Drive/NN&RL/test"
os.chdir(DATA)
features1=[]
features2=[]
labels=[]
count=0
for img in os.listdir(DATA):
  if os.path.isfile(img):
    if count<90  :
      print(img)
      orig_img=load_city_image(img)
      for i in range(32):
        features1.append(Patch_0(orig_img,Label_size)[i])
        features2.append(Patch(orig_img,PATCH1_SIZE,NUMBER1,Label_size)[i])
        labels.append(Patch_label(orig_img,PATCH2_SIZE,NUMBER2,Label_size)[i])
    count=count+1
labels=np.array(labels)
  

def pre_processing(images):
    newImages = []

    for image in images:
        newImage = skimage.transform.resize(image, (224, 224), mode='constant')
        newImages.append(newImage)
        
    return newImages
        


Copy of bea1.jpg
(256, 256)
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PI

In [6]:
np.shape(labels)



(32, 8, 8, 3)

In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import tensorlayer as tl
import skimage.transform

features1_post = np.array(pre_processing(features1))
features2_post = np.array(pre_processing(features2))

/usr/local/lib/python2.7/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [8]:
#if training
labels = np.reshape(labels, (2880,192))

#if testing
#labels = np.reshape(labels, (32,192))
labels = labels/255
print(labels.dtype)

float64


In [0]:

def cnn_model_fn(features1, features2, labels,features_orig,n,restore=True):

#         input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    epochs=11
    batch_size=32
    num_batches= int(n/batch_size)

    #if restore==True: batch_size=1
    
    X1 = tf.placeholder(tf.float32, [batch_size, 224, 224, 3])
    X2 = tf.placeholder(tf.float32, [batch_size, 224, 224, 3])
    y=tf.placeholder(tf.float32, [batch_size, 192])
    # get VGG without the last layer
    
    vgg1 = tl.models.VGG16(X1, end_with='fc2_relu')
    
    vgg2 = tl.models.VGG16(X2, end_with='fc2_relu', reuse = True)
    
     #     restore pre-trained VGG parameters`
    sess1 = tf.InteractiveSession()
    vgg1.restore_params(sess1)
    vgg_concat = tf.concat([vgg1.outputs, vgg2.outputs],1)
    
    # add one more layer
    net = tl.layers.InputLayer(vgg_concat, name='input')
    prediction = tl.layers.DenseLayer(net, 192, act=tf.nn.sigmoid, name='out')
    
    print((prediction.outputs).get_shape())
    
    saver = tf.train.Saver()
    if restore==False:
        loss_vec = []
        for i in range(0,batch_size):
            loss=0.0
            loss = tf.losses.mean_squared_error(labels[i], prediction.outputs[i])
            loss_vec.append(loss)

    #     loss=0.0
    #         for j in range(0, 128):
    # #             print(label)
    #             loss += tf.losses.mean_squared_error(labels[i][j], prediction.outputs[i][j])
        loss_op = tf.reduce_mean( loss_vec  , name='MSE' ) 
        print(loss_op)
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_op = optimizer.minimize(loss_op)


    #     x_axis = list(range(0, epochs))
        acc_mean, loss_mean = [], []

      
        init=tf.global_variables_initializer()
        with tf.Session() as sess:
            init.run()
            saver.restore(sess, "/content/drive/My Drive/NN&RL/test/model_dir_l_20/model.ckpt")
            for i in range(epochs):

                acc, los=[], []
                for j in range(num_batches):

                    x1_arr = features1[(j*batch_size):((j+1)*batch_size),:,:,:]
                    x2_arr = features2[(j*batch_size):((j+1)*batch_size),:,:,:]
                    y_arr = labels[(j*batch_size):((j+1)*batch_size),:]
                    p,l, _ = sess.run([prediction.outputs, loss_op, train_op], feed_dict={X1: x1_arr, X2: x2_arr, y: y_arr})
    #                 print("pred:" , p)
                    los.append(l)
    # #                 acc_train1 = accuracy1.eval(feed_dict={X: X_batch, y: y_batch})
    #                 print("Train accuracy-level1:", acc_train)
    #                 acc1.append(acc_train)
    #             acc_mean.append(sum(acc) / len(acc))
                loss_mean.append(sum(los) / len(los))
                print("epoch:",i)
                print(loss_mean)
                if(i==10):
                  saver.save(sess, "/content/drive/My Drive/NN&RL/test/model_dir_l_30/model.ckpt")
    else:
        #tf.reset_default_graph()
        loss_vec = []
        for i in range(0,batch_size):
            loss=0.0
            loss = tf.losses.mean_squared_error(labels[i], prediction.outputs[i])
            loss_vec.append(loss)

    #     loss=0.0
    #         for j in range(0, 128):
    # #             print(label)
    #             loss += tf.losses.mean_squared_error(labels[i][j], prediction.outputs[i][j])
        loss_op = tf.reduce_mean( loss_vec  , name='MSE' ) 
        with tf.Session() as sess:
            saver.restore(sess, "/content/drive/My Drive/NN&RL/test/model_dir_l_30/model.ckpt")
            print("Model restored.")
            result=[]
            for i in range(n):
                x1_arr = features1[i:(i+batch_size),:,:,:]
                x2_arr = features2[i:(i+batch_size),:,:,:]

                p,l = sess.run([prediction.outputs,loss_op], feed_dict={X1: x1_arr, X2: x2_arr})
                #print(p[0].shape)
                #print(p.shape)
                #print(l)
                #print(len(p))
                #temp=np.
                #for i in range(len(p)):
                #  temp[i,:,:]= p[i]
   
                #p=np.array(p)
                #p=np.transpose(p,(1, 2,0))
                #print(p)
                #print("pre:",p)
            
            temp=np.reshape(p,(32,8,8,3))
            temp=temp*255
            full_image=np.ones((128,80,3))
            #orig=(tf.image.rgb_to_grayscale(features_orig[0][:,:,:])).eval()
            full_image[:,8:72,:]=features_orig[0][:,:,:]
            for k in range(2):
                for i in range(0,128,8):
                    a2=temp[k*16+(i//8),:,:,:]
                    #print(a)
                    print(a2.shape)
                    full_image[i:i+8,0+k*72:8+k*72,:]=temp[k*16+(i//8),:,:,:]
                   
            final_result=Image.fromarray(np.uint8(full_image))
            final_result.save("/content/drive/My Drive/NN&RL/final_img.png","PNG")
            print("Saved image")
            

In [10]:
#if training
cnn_model_fn(features1_post, features2_post, labels,features1,2880,False)
#if testing
#cnn_model_fn(features1_post, features2_post, labels,features1,1,True)

[TL] InputLayer  vgg16/input: (32, 224, 224, 3)
[TL] Conv2d vgg16/conv1_1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d vgg16/conv1_2: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d vgg16/pool1: filter_size: (2, 2) strides: (2, 2) padding: SAME
[TL] Conv2d vgg16/conv2_1: n_filter: 128 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d vgg16/conv2_2: n_filter: 128 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d vgg16/pool2: filter_size: (2, 2) strides: (2, 2) padding: SAME
[TL] Conv2d vgg16/conv3_1: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d vgg16/conv3_2: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d vgg16/conv3_3: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d vgg16/pool3: filter_size: (2, 2) strides: (2, 2) padding: SAME
[TL] Conv2d vgg16/conv4_1: n_filter: 512 